In [200]:
#import all relevant packages
import torch, torchvision
import torchvision.transforms as transforms
import numpy as np
from backpack import backpack, extend
from backpack.extensions import DiagHessian, KFAC
import matplotlib.pyplot as plt
from torch.distributions.multivariate_normal import MultivariateNormal
from torch.distributions.normal import Normal
from torch.nn import functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, random_split
from torch.autograd import Variable
from scipy.stats import pearsonr
from mpl_toolkits.axes_grid1 import AxesGrid
import mpl_toolkits as mpl
from scipy.stats import chi2, norm
import random
#import warnings
#warnings.filterwarnings("ignore")

plt.rcParams["mpl_toolkits.legacy_colorbar"] = False

s=2
np.random.seed(s)
torch.manual_seed(s)
torch.cuda.manual_seed(s)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

#plt stuff
plt.rcParams["figure.figsize"] = (20,10)

In [253]:
def NN():
    
    features = torch.nn.Sequential(
        torch.nn.Linear(1, 1)
        #torch.nn.Linear(1, 1)
    )
    return(features)

net = NN()
#loss_function = torch.nn.MSELoss()

#train_optimizer = torch.optim.Adam(net.parameters(), lr=1e-3, weight_decay=5e-4)

In [242]:
#print(list(net.parameters()))

In [254]:
def criterion(out, label):
    return (label - out)**2
loss_func = torch.nn.MSELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.5)

In [255]:
#___Create Dataset____
#Network (Ax + b) should learn A=A_size, b=0 
A_size = 3
data = []

for i in range(1, 11):
    data.append((i, i*A_size))
print(data)# = [(1,3), (2,6), (3,9), (4,12), (5,15), (6,18)]

[(1, 3), (2, 6), (3, 9), (4, 12), (5, 15), (6, 18), (7, 21), (8, 24), (9, 27), (10, 30)]


In [262]:
def trainloop():
    for epoch in range(100):
        for i, data2 in enumerate(data):
            X, Y = iter(data2)
            X, Y = Variable(torch.FloatTensor([X]), requires_grad=True), Variable(torch.FloatTensor([Y]), requires_grad=False)
            optimizer.zero_grad()
            outputs = net(X)
            loss = criterion(outputs, Y)
            loss.backward()
            optimizer.step()
            if (i % 10 == 0):
                print("Epoch {} - loss: {}".format(epoch, loss.data[0]))

In [263]:
trainloop()

Epoch 0 - loss: 11.498779296875
Epoch 1 - loss: 0.47282543778419495
Epoch 2 - loss: 0.16993378102779388
Epoch 3 - loss: 0.174896240234375
Epoch 4 - loss: 0.15049239993095398
Epoch 5 - loss: 0.13298319280147552
Epoch 6 - loss: 0.11703185737133026
Epoch 7 - loss: 0.103058822453022
Epoch 8 - loss: 0.09074508398771286
Epoch 9 - loss: 0.07990328222513199
Epoch 10 - loss: 0.0703572928905487
Epoch 11 - loss: 0.0619516521692276
Epoch 12 - loss: 0.05454998090863228
Epoch 13 - loss: 0.04803291708230972
Epoch 14 - loss: 0.0422942154109478
Epoch 15 - loss: 0.03724120929837227
Epoch 16 - loss: 0.03279190510511398
Epoch 17 - loss: 0.02887425385415554
Epoch 18 - loss: 0.025424623861908913
Epoch 19 - loss: 0.02238694578409195
Epoch 20 - loss: 0.019712409004569054
Epoch 21 - loss: 0.017357399687170982
Epoch 22 - loss: 0.015283740125596523
Epoch 23 - loss: 0.013457589782774448
Epoch 24 - loss: 0.01185003574937582
Epoch 25 - loss: 0.01043412834405899
Epoch 26 - loss: 0.009187495335936546
Epoch 27 - loss:

In [264]:
print(list(net.parameters()))

[Parameter containing:
tensor([[2.9999]], requires_grad=True), Parameter containing:
tensor([0.0010], requires_grad=True)]


In [210]:
print(net(Variable(torch.Tensor([[[1]]]))))

tensor([[[3.0028]]], grad_fn=<AddBackward0>)


In [249]:
def get_Hessian_NN(model, train_loader, prec0, device='cpu', verbose=True):
    lossfunc = criterion()

    extend(lossfunc, debug=False)
    extend(model, debug=False)

    Cov_diag = []
    for param in model.parameters():
        ps = param.size()
        print("parameter size: ", ps)
        Cov_diag.append(torch.zeros(ps, device=device))
        #print(param.numel())

    #var0 = 1/prec0
    max_len = len(train_loader)

    with backpack(DiagHessian()):

        for batch_idx, (x, y) in enumerate(train_loader):

            if device == 'cuda':
                x, y = x.float().cuda(), y.long().cuda()
            
            #y = y.view(-1, 1)
            #y = y.float()
            #print(y)
            #x = x.view(-1, 1)

            model.zero_grad()
            lossfunc(model(x), y).backward()

            with torch.no_grad():
                # Hessian of weight
                for idx, param in enumerate(model.parameters()):

                    H_ = param.diag_h
                    #add prior here
                    H_ += prec0 * torch.ones(H_.size())
                    H_inv = torch.sqrt(1/H_) #<-- standard deviation
                    #H_inv = 1/H_              #<-- variance 

                    rho = 1-1/(batch_idx+1)

                    Cov_diag[idx] = rho*Cov_diag[idx] + (1-rho)* H_inv
            
            if verbose:
                print("Batch: {}/{}".format(batch_idx, max_len))
    
    return(Cov_diag)

In [250]:
perceptronDiagH = get_Hessian_NN(net, train_loader=data, prec0=0.0001)
print(perceptronDiagH)

TypeError: criterion() missing 2 required positional arguments: 'out' and 'label'

In [191]:
class NumbersDataset(Dataset):
    def __init__(self):
        self.nums = torch.tensor(np.linspace(-1, 1, 500))#torch.rand(-1, 1, 500)
        self.samples = []
        #self.samples = self.nums.clone()
        #self.samples = torch.tensor([self.nums.clone(), self.nums.clone()]).view(2, -1)
        self.w = 1
        self.b = 1.3
        for x in self.nums:
            self.samples.append((x, x))# self.w * x + self.b))
            #if x > 0:
            #    self.samples.append((x, 1))
            #else:
            #    self.samples.append((x, 0))
        

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        #return self.samples[idx]
        num, label = self.samples[idx]
        return num, label
        #$return self.samples[idx]
if __name__ == '__main__':
    dataset = NumbersDataset()
    trainset, testset = random_split(dataset, [400, 100])
    dataloader = DataLoader(trainset, batch_size=10, shuffle=True, num_workers=2)
    testloader = DataLoader(testset, batch_size=5, shuffle=False, num_workers=2)
    #print(len(dataset))
    #print(dataset[99])
    #print(dataset[0])
    for i, batch in enumerate(dataloader):
        print(i, batch)
        if i > 10:
            break
        
#trainset, testset = random_split(dataset, [400, 100])

#dataloader = DataLoader(trainset, batch_size=5, shuffle=True, num_workers=2)
#testloader = DataLoader(testset, batch_size=5, shuffle=False, num_workers=2)

0 [tensor([-0.9078,  0.7275,  0.0220,  0.1062, -0.6072,  0.8116,  0.7194, -0.2064,
        -0.2425,  0.3387], dtype=torch.float64), tensor([-0.9078,  0.7275,  0.0220,  0.1062, -0.6072,  0.8116,  0.7194, -0.2064,
        -0.2425,  0.3387], dtype=torch.float64)]
1 [tensor([ 0.3908, -0.7114,  0.1222,  0.1303,  0.0902, -0.6353, -0.4389, -0.7916,
        -0.7395, -0.5030], dtype=torch.float64), tensor([ 0.3908, -0.7114,  0.1222,  0.1303,  0.0902, -0.6353, -0.4389, -0.7916,
        -0.7395, -0.5030], dtype=torch.float64)]
2 [tensor([ 0.6393, -0.5992,  0.7074,  0.0982, -0.3828, -0.0341, -0.4269, -0.6313,
         0.2786, -0.5150], dtype=torch.float64), tensor([ 0.6393, -0.5992,  0.7074,  0.0982, -0.3828, -0.0341, -0.4269, -0.6313,
         0.2786, -0.5150], dtype=torch.float64)]
3 [tensor([-0.7756,  0.7355,  0.4870,  0.6232,  0.0461,  0.1743, -0.5190, -0.7635,
         0.6834,  0.9639], dtype=torch.float64), tensor([-0.7756,  0.7355,  0.4870,  0.6232,  0.0461,  0.1743, -0.5190, -0.7635,
     

In [192]:
def get_accuracy(output, targets):
    """Helper function to print the accuracy"""
    predictions = output.argmax(dim=1, keepdim=True).view_as(targets)
    return predictions.eq(targets).float().mean().item()

In [193]:
net = net.float()
def train(verbose=False, num_iter=30):
    max_len = len(dataloader)
    for iter in range(num_iter):
        for batch_idx, (x, y) in enumerate(dataloader):
            y = y.view(-1, 1)
            y = y.float()
            #print(y)
            x = x.view(-1, 1)
            #print(x)
            output = net(x.float())
            #print(output)
            #output_flat = output.view(-1)
            #y_flat = y.view(-1)

            accuracy = get_accuracy(output, y)

            loss = loss_function(output, y)
            loss.backward()
            train_optimizer.step()
            train_optimizer.zero_grad()

            if verbose:
                if batch_idx % 10 == 0:
                    print(
                        "Iteration {}; {}/{} \t".format(iter, batch_idx, max_len) +
                        "Minibatch Loss %.3f  " % (loss) +
                        "Accuracy %.0f" % (accuracy * 100) + "%"
                    )

    #print("saving model at: {}".format(MNIST_PATH))
    #torch.save(mnist_model.state_dict(), MNIST_PATH)

In [194]:
train(verbose=True)

Iteration 0; 0/40 	Minibatch Loss 2.050  Accuracy 0%
Iteration 0; 10/40 	Minibatch Loss 1.324  Accuracy 0%
Iteration 0; 20/40 	Minibatch Loss 1.633  Accuracy 0%
Iteration 0; 30/40 	Minibatch Loss 1.168  Accuracy 0%
Iteration 1; 0/40 	Minibatch Loss 1.345  Accuracy 0%
Iteration 1; 10/40 	Minibatch Loss 1.551  Accuracy 0%
Iteration 1; 20/40 	Minibatch Loss 1.061  Accuracy 0%
Iteration 1; 30/40 	Minibatch Loss 1.396  Accuracy 0%
Iteration 2; 0/40 	Minibatch Loss 1.331  Accuracy 0%
Iteration 2; 10/40 	Minibatch Loss 0.995  Accuracy 0%
Iteration 2; 20/40 	Minibatch Loss 1.182  Accuracy 0%
Iteration 2; 30/40 	Minibatch Loss 1.788  Accuracy 0%
Iteration 3; 0/40 	Minibatch Loss 0.974  Accuracy 0%
Iteration 3; 10/40 	Minibatch Loss 0.886  Accuracy 0%
Iteration 3; 20/40 	Minibatch Loss 1.330  Accuracy 0%
Iteration 3; 30/40 	Minibatch Loss 1.411  Accuracy 0%
Iteration 4; 0/40 	Minibatch Loss 0.981  Accuracy 0%
Iteration 4; 10/40 	Minibatch Loss 1.654  Accuracy 0%
Iteration 4; 20/40 	Minibatch Los

In [195]:
for (x, y) in testloader:
            y = y.view(-1, 1)
            y = y.float()
            #print(y)
            x = x.view(-1, 1)
            #print(x)
            output = net(x.float())
            #print(output)
            #output_flat = output.view(-1)
            #y_flat = y.view(-1)

            accuracy = get_accuracy(output, y)
            print(x)
            print(output)
            print(accuracy)
            print()

tensor([[ 0.1423],
        [-0.3307],
        [-0.2545],
        [-0.1423],
        [ 0.4749]], dtype=torch.float64)
tensor([[ 0.4302],
        [-0.9884],
        [-0.7599],
        [-0.4233],
        [ 1.4280]], grad_fn=<AddmmBackward>)
0.0

tensor([[ 0.6353],
        [-0.2745],
        [-0.6232],
        [ 0.2064],
        [ 0.3747]], dtype=torch.float64)
tensor([[ 1.9088],
        [-0.8201],
        [-1.8659],
        [ 0.6225],
        [ 1.1274]], grad_fn=<AddmmBackward>)
0.0

tensor([[-0.1543],
        [ 0.5752],
        [ 0.1503],
        [ 0.1463],
        [ 0.6313]], dtype=torch.float64)
tensor([[-0.4594],
        [ 1.7285],
        [ 0.4542],
        [ 0.4422],
        [ 1.8968]], grad_fn=<AddmmBackward>)
0.0

tensor([[-0.7355],
        [-0.3186],
        [-0.9679],
        [ 0.9238],
        [ 0.9399]], dtype=torch.float64)
tensor([[-2.2025],
        [-0.9523],
        [-2.8998],
        [ 2.7744],
        [ 2.8225]], grad_fn=<AddmmBackward>)
0.0

tensor([[ 0.2986],
        [

In [196]:
state = net.state_dict()
print(state)

OrderedDict([('0.weight', tensor([[2.9994]])), ('0.bias', tensor([0.0034]))])


In [197]:
for i, (a, b) in enumerate(dataloader):
    X = a
    y = b
    if i > 0:
        break
        
y = y.view(-1, 1)
y = y.float()
#print(y)
X = X.view(-1, 1)
#X, y = dataloader
model = extend(net)
lossfunc = extend(loss_function)
loss = lossfunc(model(X.float()), y)

with backpack(DiagHessian()):
    loss.backward()

for name, param in model.named_parameters():
    print(name)
    print(param.grad)
    print(param.diag_h)
    print(torch.sqrt(1/param.diag_h))

0.weight
tensor([[1621.3851]])
tensor([[0.7700]])
tensor([[1.1396]])
0.bias
tensor([8.2693])
tensor([2.])
tensor([0.7071])


In [199]:
test = get_Hessian_NN(model=net, train_loader=dataloader, prec0=0.0001)
print(test)

parameter size:  torch.Size([1, 1])
parameter size:  torch.Size([1])
Batch: 0/40
Batch: 1/40
Batch: 2/40
Batch: 3/40
Batch: 4/40
Batch: 5/40
Batch: 6/40
Batch: 7/40
Batch: 8/40
Batch: 9/40
Batch: 10/40
Batch: 11/40
Batch: 12/40
Batch: 13/40
Batch: 14/40
Batch: 15/40
Batch: 16/40
Batch: 17/40
Batch: 18/40
Batch: 19/40
Batch: 20/40
Batch: 21/40
Batch: 22/40
Batch: 23/40
Batch: 24/40
Batch: 25/40
Batch: 26/40
Batch: 27/40
Batch: 28/40
Batch: 29/40
Batch: 30/40
Batch: 31/40
Batch: 32/40
Batch: 33/40
Batch: 34/40
Batch: 35/40
Batch: 36/40
Batch: 37/40
Batch: 38/40
Batch: 39/40
[tensor([[1.2560]]), tensor([0.7071])]
